In [2]:
import pandas as pd

## Water Quality Dataset

In [3]:
# read csv
start_df = pd.read_csv("water_quality_2020_2021_datasd.csv")
col_to_keep = ['station', 'date_sample', 'time','parameter', 'value']
start_df = start_df[col_to_keep]
# drop entries with no data
start_df = start_df.dropna(subset=['value'])

In [4]:
# identify unique characteristics
unique_char = start_df['parameter'].unique()
to_keep = ['ENTERO', 'TEMP', 'DO', 'PH', 'CHLOROPHYLL', 'DENSITY', 'FECAL', 'SALINITY']
# pivot table based on characteristics to keep 
pivot_df = start_df.pivot_table(index=['station', 'date_sample', 'time'], columns='parameter', values='value', aggfunc='mean')
pivot_df.reset_index(inplace=True)
pivot_df = pivot_df[['station', 'date_sample', 'time'] + to_keep]
# drop any entries that are not full of data
pivot_df = pivot_df.dropna(subset=to_keep, thresh=8)

In [5]:
# pop salinity (dependent var)
pivot_df['SALINITY'] = pivot_df.pop('SALINITY')
# rename date column
pivot_df.rename(columns={'date_sample': 'Date',
                         'time': 'Time',
                         'TEMP': 'WATER_TEMP'}, inplace=True)
pivot_df.head()

parameter,station,Date,Time,ENTERO,WATER_TEMP,DO,PH,CHLOROPHYLL,DENSITY,FECAL,SALINITY
0,A1,1/13/2020,7:52:00 PST,4.0,14.443333,6.720000,7.966667,0.813333,24.972667,36.0,33.550667
1,A1,1/13/2021,7:53:00 PST,2.0,13.396667,6.290000,7.960000,1.470000,25.234667,2.0,33.609333
2,A1,1/19/2021,9:02:00 PST,2.0,14.713333,7.770000,8.086667,2.743333,24.944000,2.0,33.587000
3,A1,1/2/2020,7:44:00 PST,2.0,15.193333,7.563333,8.103333,1.500000,24.671000,2.0,33.366667
4,A1,1/21/2020,9:43:00 PST,2.0,15.130000,7.576667,8.060000,0.743333,24.819000,2.0,33.542000


## Add Weather Data

In [6]:
# change times in water pollution df to aggregate with weather data

# extract hour part from time
time_df = pd.DataFrame(pivot_df)
time_df['hour'] = pivot_df['Time'].str.split(':').str[0]
# comvert hour to integer
time_df['hour'] = time_df['hour'].astype(int)
# calculate the minutes part
minutes = time_df['Time'].str.split(':').str[1].astype(int)
# round hour to nearest hour
time_df['Rounded_Hour'] = time_df['hour'] + (minutes >= 30)
# convert back to string and add :00
time_df['Rounded_Time'] = time_df['Rounded_Hour'].astype(str) + ":00"

# drop temporary columns 
time_df.drop(['hour', 'Rounded_Hour'], axis=1, inplace=True)
# replace time with Rounded_Time
pivot_df['Time'] = time_df['Rounded_Time']
pivot_df.head()


parameter,station,Date,Time,ENTERO,WATER_TEMP,DO,PH,CHLOROPHYLL,DENSITY,FECAL,SALINITY
0,A1,1/13/2020,8:00,4.0,14.443333,6.720000,7.966667,0.813333,24.972667,36.0,33.550667
1,A1,1/13/2021,8:00,2.0,13.396667,6.290000,7.960000,1.470000,25.234667,2.0,33.609333
2,A1,1/19/2021,9:00,2.0,14.713333,7.770000,8.086667,2.743333,24.944000,2.0,33.587000
3,A1,1/2/2020,8:00,2.0,15.193333,7.563333,8.103333,1.500000,24.671000,2.0,33.366667
4,A1,1/21/2020,10:00,2.0,15.130000,7.576667,8.060000,0.743333,24.819000,2.0,33.542000


In [7]:
# upload weather data
weather_df = pd.read_csv('weather_data.csv')
# combine with water pollution dataset
full_df = pd.merge(pivot_df, weather_df, on=['Date', 'Time'], how='left')

# print statistics of data
print("Length = ", len(full_df))
full_df.head()

Length =  1743


,station,Date,Time,ENTERO,WATER_TEMP,DO,PH,CHLOROPHYLL,DENSITY,FECAL,SALINITY,AIR_TEMP,HUMIDITY,WINDSPEED,CLOUD_COVER,SOLAR_RADIATION
0,A1,1/13/2020,8:00,4.0,14.443333,6.720000,7.966667,0.813333,24.972667,36.0,33.550667,52.2,74.59,3.4,23.3,89.0
1,A1,1/13/2021,8:00,2.0,13.396667,6.290000,7.960000,1.470000,25.234667,2.0,33.609333,53.9,40.57,2.5,75.3,108.0
2,A1,1/19/2021,9:00,2.0,14.713333,7.770000,8.086667,2.743333,24.944000,2.0,33.587000,61.1,65.74,1.9,75.3,193.0
3,A1,1/2/2020,8:00,2.0,15.193333,7.563333,8.103333,1.500000,24.671000,2.0,33.366667,52.0,81.85,0.0,58.5,85.0
4,A1,1/21/2020,10:00,2.0,15.130000,7.576667,8.060000,0.743333,24.819000,2.0,33.542000,56.6,89.54,5.7,71.3,234.0


In [24]:
# extract dataset
full_df.to_csv('water_quality_and_weather.csv', index=False)